In [23]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [25]:
tf.keras.backend.clear_session()

In [26]:
subject = 'Ceará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [27]:
data = pd.read_csv('2003_mo_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Ceará - Consumo de Cimento (t)
0,2003-1,8.274459,0.301376,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.684040,6.965352e+07,4.525724e+06,7.532751,6.293821e+07,107.673000,81.265
1,2003-2,8.268512,0.301393,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.684476,6.974354e+07,4.530359e+06,7.535808,6.297178e+07,107.237962,65.200
2,2003-3,8.262564,0.301309,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.684912,6.983356e+07,4.534994e+06,7.538865,6.300536e+07,108.742184,55.165
3,2003-4,8.256617,0.300917,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.685347,6.992358e+07,4.539629e+06,7.541921,6.303893e+07,109.391112,53.928
4,2003-5,8.250670,0.300417,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.685783,7.001361e+07,4.544265e+06,7.544978,6.307250e+07,111.602457,65.334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.535686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.122997,186.918
236,2022-9,NaN,0.533990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.220173,179.828
237,2022-10,NaN,0.530449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237.879214,174.940
238,2022-11,NaN,0.526253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.799325,182.597


In [28]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [29]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      75.575
1      53.065
2      67.755
3      64.858
4      72.377
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Ceará - Consumo de Cimento (t), Length: 240, dtype: float64

In [30]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.423186,-0.021469,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.409954,1.190773,-0.838130,0.874129,1.017258,1.523635
158,1.399318,-0.031468,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.414132,1.193287,-0.881990,0.871719,1.015919,1.541381
159,1.375451,-0.044253,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.418310,1.195800,-0.925849,0.869309,1.014581,1.523536
160,1.351584,-0.046806,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422488,1.198313,-0.969709,0.866899,1.013242,1.507028


In [31]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       75.575
1       53.065
2       67.755
3       64.858
4       72.377
        ...   
157    134.739
158    151.819
159    124.858
160    162.684
161    169.251
Name: Ceará - Consumo de Cimento (t), Length: 162, dtype: float64

In [32]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [33]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [34]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
126,0.941464,1.223482,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.365676,0.843485,0.634116,1.037548,1.010785,0.074200
127,0.969984,1.232594,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.371209,0.863067,0.604648,1.047639,1.024388,0.100668
128,0.998504,1.233295,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.376742,0.882650,0.575180,1.057730,1.037991,0.116036
129,1.027024,1.227338,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.382275,0.902233,0.545712,1.067821,1.051594,0.134124
130,1.055544,1.232230,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.387808,0.921816,0.516244,1.077913,1.065197,0.158476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [35]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [36]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [37]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(360, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [38]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [39]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4021506022, 2340443190, 3916124845, 2328880335, 1807330442, 1012313164, 2374032038, 1287800125, 3256228706, 2836881329, 2831786496, 1905160376, 4105858773, 3968989179, 3278091742, 2931343595, 470402549, 1052015966, 2918500265, 3354196462, 171033838, 978525541, 4172131093, 928881327, 3149986028, 1186887714, 608969191, 1158940938, 1467541039, 2544610892, 1021380682, 915082630, 3763469323, 687650603, 461518357, 2375699042, 1633762250, 2830207011, 2020064961, 161475397, 2491542367, 2480652391, 2298139837, 3743044049, 3673591933, 3558889922, 3916228985, 3961048295, 2437954737, 2432075856]


Step: 0 ___________________________________________
val_loss: 159.46795654296875
winner_seed: 4021506022


Step: 1 ___________________________________________
val_loss: 213.14715576171875


Step: 2 ___________________________________________
val_loss: 189.45103454589844


Step: 3 ___________________________________________
val_loss: 132.68682861328125
winner_seed: 2328880335


Step: 4 __________________

In [40]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 4s 187ms/step - loss: 22515.0000 - val_loss: 7312.6582
Epoch 2/10000
4/4 [==============================] - 0s 16ms/step - loss: 4207.3350 - val_loss: 2185.0261
Epoch 3/10000
4/4 [==============================] - 0s 19ms/step - loss: 1913.2545 - val_loss: 1793.3802
Epoch 4/10000
4/4 [==============================] - 0s 16ms/step - loss: 2460.3127 - val_loss: 4733.6357
Epoch 5/10000
4/4 [==============================] - 0s 17ms/step - loss: 2588.1511 - val_loss: 716.7956
Epoch 6/10000
4/4 [==============================] - 0s 15ms/step - loss: 786.7145 - val_loss: 667.0139
Epoch 7/10000
4/4 [==============================] - 0s 15ms/step - loss: 773.6739 - val_loss: 669.5858
Epoch 8/10000
4/4 [==============================] - 0s 15ms/step - loss: 772.1148 - val_loss: 663.7749
Epoch 9/10000
4/4 [==============================] - 0s 15ms/step - loss: 764.9711 - val_loss: 641.7292
Epoch 10/10000
4/4 [==============================] -

4/4 [==============================] - 0s 18ms/step - loss: 766.0369 - val_loss: 624.2659
Epoch 80/10000
4/4 [==============================] - 0s 18ms/step - loss: 755.4656 - val_loss: 651.1185
Epoch 81/10000
4/4 [==============================] - 0s 19ms/step - loss: 758.5835 - val_loss: 627.4524
Epoch 82/10000
4/4 [==============================] - 0s 19ms/step - loss: 761.5695 - val_loss: 672.5770
Epoch 83/10000
4/4 [==============================] - 0s 20ms/step - loss: 761.5361 - val_loss: 673.0296
Epoch 84/10000
4/4 [==============================] - 0s 19ms/step - loss: 764.3120 - val_loss: 670.3383
Epoch 85/10000
4/4 [==============================] - 0s 20ms/step - loss: 763.9915 - val_loss: 663.7973
Epoch 86/10000
4/4 [==============================] - 0s 20ms/step - loss: 762.1328 - val_loss: 641.2225
Epoch 87/10000
4/4 [==============================] - 0s 21ms/step - loss: 755.3177 - val_loss: 659.1904
Epoch 88/10000
4/4 [==============================] - 0s 21ms/step - l

4/4 [==============================] - 0s 20ms/step - loss: 756.4270 - val_loss: 691.3000
Epoch 157/10000
4/4 [==============================] - 0s 20ms/step - loss: 769.8126 - val_loss: 687.2185
Epoch 158/10000
4/4 [==============================] - 0s 20ms/step - loss: 765.0349 - val_loss: 652.1166
Epoch 159/10000
4/4 [==============================] - 0s 19ms/step - loss: 760.3171 - val_loss: 623.5012
Epoch 160/10000
4/4 [==============================] - 0s 17ms/step - loss: 754.8973 - val_loss: 652.1327
Epoch 161/10000
4/4 [==============================] - 0s 20ms/step - loss: 761.9551 - val_loss: 634.7291
Epoch 162/10000
4/4 [==============================] - 0s 21ms/step - loss: 763.2449 - val_loss: 639.4227
Epoch 163/10000
4/4 [==============================] - 0s 21ms/step - loss: 764.5660 - val_loss: 712.8537
Epoch 164/10000
4/4 [==============================] - 0s 20ms/step - loss: 771.9844 - val_loss: 647.9198
Epoch 165/10000
4/4 [==============================] - 0s 16ms

4/4 [==============================] - 0s 18ms/step - loss: 763.8832 - val_loss: 641.4232
Epoch 234/10000
4/4 [==============================] - 0s 18ms/step - loss: 755.5270 - val_loss: 675.5754
Epoch 235/10000
4/4 [==============================] - 0s 18ms/step - loss: 761.0200 - val_loss: 616.3752
Epoch 236/10000
4/4 [==============================] - 0s 19ms/step - loss: 761.8017 - val_loss: 666.4537
Epoch 237/10000
4/4 [==============================] - 0s 17ms/step - loss: 757.9048 - val_loss: 637.1863
Epoch 238/10000
4/4 [==============================] - 0s 18ms/step - loss: 755.1169 - val_loss: 633.9786
Epoch 239/10000
4/4 [==============================] - 0s 17ms/step - loss: 760.9428 - val_loss: 628.5925
Epoch 240/10000
4/4 [==============================] - 0s 18ms/step - loss: 757.3688 - val_loss: 614.9739
Epoch 241/10000
4/4 [==============================] - 0s 19ms/step - loss: 757.7900 - val_loss: 639.7369
Epoch 242/10000
4/4 [==============================] - 0s 18ms

4/4 [==============================] - 0s 18ms/step - loss: 754.7124 - val_loss: 628.1526
Epoch 311/10000
4/4 [==============================] - 0s 17ms/step - loss: 755.4980 - val_loss: 633.7981
Epoch 312/10000
4/4 [==============================] - 0s 18ms/step - loss: 752.6368 - val_loss: 654.8654
Epoch 313/10000
4/4 [==============================] - 0s 19ms/step - loss: 754.8017 - val_loss: 642.1718
Epoch 314/10000
4/4 [==============================] - 0s 17ms/step - loss: 752.5579 - val_loss: 620.6744
Epoch 315/10000
4/4 [==============================] - 0s 17ms/step - loss: 759.6865 - val_loss: 618.1121
Epoch 316/10000
4/4 [==============================] - 0s 18ms/step - loss: 760.0145 - val_loss: 629.6656
Epoch 317/10000
4/4 [==============================] - 0s 19ms/step - loss: 746.4171 - val_loss: 637.5383
Epoch 318/10000
4/4 [==============================] - 0s 17ms/step - loss: 764.1473 - val_loss: 689.0712
Epoch 319/10000
4/4 [==============================] - 0s 18ms

...

4/4 [==============================] - 0s 20ms/step - loss: 121.3346 - val_loss: 256.4952
Epoch 1691/10000
4/4 [==============================] - 0s 19ms/step - loss: 72.6646 - val_loss: 270.2369
Epoch 1692/10000
4/4 [==============================] - 0s 21ms/step - loss: 75.8583 - val_loss: 257.5552
Epoch 1693/10000
4/4 [==============================] - 0s 20ms/step - loss: 75.2086 - val_loss: 443.7862
Epoch 1694/10000
4/4 [==============================] - 0s 19ms/step - loss: 118.0019 - val_loss: 303.5999
Epoch 1695/10000
4/4 [==============================] - 0s 19ms/step - loss: 112.7823 - val_loss: 285.2853
Epoch 1696/10000
4/4 [==============================] - 0s 20ms/step - loss: 99.1456 - val_loss: 292.9530
Epoch 1697/10000
4/4 [==============================] - 0s 20ms/step - loss: 88.4961 - val_loss: 278.6985
Epoch 1698/10000
4/4 [==============================] - 0s 17ms/step - loss: 68.3508 - val_loss: 250.8025
Epoch 1699/10000
4/4 [==============================] - 0s 1

4/4 [==============================] - 0s 15ms/step - loss: 72.8721 - val_loss: 272.2823
Epoch 1768/10000
4/4 [==============================] - 0s 15ms/step - loss: 68.4317 - val_loss: 274.2657
Epoch 1769/10000
4/4 [==============================] - 0s 18ms/step - loss: 54.5497 - val_loss: 252.5156
Epoch 1770/10000
4/4 [==============================] - 0s 15ms/step - loss: 51.8675 - val_loss: 245.0667
Epoch 1771/10000
4/4 [==============================] - 0s 16ms/step - loss: 57.4338 - val_loss: 227.7730
Epoch 1772/10000
4/4 [==============================] - 0s 15ms/step - loss: 59.9054 - val_loss: 220.5376
Epoch 1773/10000
4/4 [==============================] - 0s 17ms/step - loss: 63.1441 - val_loss: 228.9259
Epoch 1774/10000
4/4 [==============================] - 0s 16ms/step - loss: 59.3511 - val_loss: 224.9126
Epoch 1775/10000
4/4 [==============================] - 0s 15ms/step - loss: 50.7214 - val_loss: 256.4845
Epoch 1776/10000
4/4 [==============================] - 0s 16ms

4/4 [==============================] - 0s 18ms/step - loss: 25.6663 - val_loss: 216.4078
Epoch 1845/10000
4/4 [==============================] - 0s 20ms/step - loss: 21.4905 - val_loss: 263.5457
Epoch 1846/10000
4/4 [==============================] - 0s 20ms/step - loss: 21.0251 - val_loss: 229.7434
Epoch 1847/10000
4/4 [==============================] - 0s 19ms/step - loss: 25.8837 - val_loss: 225.9271
Epoch 1848/10000
4/4 [==============================] - 0s 19ms/step - loss: 22.2828 - val_loss: 218.1425
Epoch 1849/10000
4/4 [==============================] - 0s 19ms/step - loss: 20.9647 - val_loss: 214.0034
Epoch 1850/10000
4/4 [==============================] - 0s 17ms/step - loss: 20.6791 - val_loss: 214.8880
Epoch 1851/10000
4/4 [==============================] - 0s 17ms/step - loss: 24.9793 - val_loss: 235.5913
Epoch 1852/10000
4/4 [==============================] - 0s 18ms/step - loss: 24.7409 - val_loss: 235.8593
Epoch 1853/10000
4/4 [==============================] - 0s 19ms

4/4 [==============================] - 0s 17ms/step - loss: 13.1863 - val_loss: 240.7754
Epoch 1922/10000
4/4 [==============================] - 0s 17ms/step - loss: 11.5846 - val_loss: 223.5600
Epoch 1923/10000
4/4 [==============================] - 0s 18ms/step - loss: 12.1554 - val_loss: 227.2776
Epoch 1924/10000
4/4 [==============================] - 0s 17ms/step - loss: 12.6274 - val_loss: 218.9829
Epoch 1925/10000
4/4 [==============================] - 0s 18ms/step - loss: 11.5267 - val_loss: 219.8008
Epoch 1926/10000
4/4 [==============================] - 0s 19ms/step - loss: 16.1448 - val_loss: 223.0257
Epoch 1927/10000
4/4 [==============================] - 0s 19ms/step - loss: 11.0303 - val_loss: 218.6312
Epoch 1928/10000
4/4 [==============================] - 0s 20ms/step - loss: 13.9292 - val_loss: 227.5203
Epoch 1929/10000
4/4 [==============================] - 0s 20ms/step - loss: 10.1607 - val_loss: 215.8431
Epoch 1930/10000
4/4 [==============================] - 0s 21ms

4/4 [==============================] - 0s 21ms/step - loss: 7.5909 - val_loss: 232.5144
Epoch 1999/10000
4/4 [==============================] - 0s 20ms/step - loss: 13.2586 - val_loss: 220.3746
Epoch 2000/10000
4/4 [==============================] - 0s 18ms/step - loss: 10.0595 - val_loss: 219.7190
Epoch 2001/10000
4/4 [==============================] - 0s 19ms/step - loss: 10.4775 - val_loss: 236.7037
Epoch 2002/10000
4/4 [==============================] - 0s 19ms/step - loss: 5.7304 - val_loss: 242.4616
Epoch 2003/10000
4/4 [==============================] - 0s 18ms/step - loss: 9.1577 - val_loss: 265.2416
Epoch 2004/10000
4/4 [==============================] - 0s 18ms/step - loss: 8.0255 - val_loss: 237.8735
Epoch 2005/10000
4/4 [==============================] - 0s 19ms/step - loss: 10.2938 - val_loss: 234.4767
Epoch 2006/10000
4/4 [==============================] - 0s 19ms/step - loss: 7.7544 - val_loss: 240.1160
Epoch 2007/10000
4/4 [==============================] - 0s 21ms/step

In [41]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [42]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,165.164383,162.519012,164.257324,165.661713,163.497971,164.381805,164.045837,153.910904,146.172455,150.84787,153.319885,143.998718,146.987122,160.061737,136.607025,143.93924,146.506561,168.919418,149.766953,130.626114,144.812439,147.608551,148.952621,149.966766,150.636414,151.108917,151.515717,151.79509,151.489807,151.263016
Target,179.243,194.134,186.948,194.907,179.505,173.278,160.261,113.987,114.289,117.313,110.502,147.996,152.386,177.259,155.913,172.41,179.419,157.165,151.601,129.537,105.236,110.835,127.925,127.138,157.128,164.44,140.298,168.407,163.615,155.07
Error,14.078613,31.61499,22.690674,29.245285,16.007034,8.896194,3.784836,39.923904,31.883453,33.534866,42.817886,3.997284,5.39888,17.197266,19.305969,28.470764,32.912445,11.754425,1.834045,1.089111,39.576439,36.773552,21.027618,22.828766,6.491592,13.331085,11.217712,16.611908,12.125198,3.806992


In [43]:
display(mae)
display(mape)

19.340956

0.13860321

In [44]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [45]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined